In [18]:
import sys
sys.path.append('../LegalBot')

from RAG_v1 import RAG_Bot

import numpy as np
import pandas as pd
from tqdm import tqdm

from ragas.metrics import faithfulness, answer_relevancy, FaithulnesswithHHEM, context_precision, context_recall
from nltk.translate.bleu_score import sentence_bleu
from ragas import evaluate
from datasets import Dataset 

In [9]:
# collection_names = ['Uk', 'Wales', 'NothernIreland', 'Scotland']
# bot = RAG_Bot(collection_names=collection_names, text_splitter='SpaCy', embedding_model="SentenceTransformers")

Cluster_URL: https://gwqaighsqp6gtrk8kwmnmg.c0.us-west3.gcp.weaviate.cloud, Cluster_API: 2xKmTTGUawzXYItFrMR9JgTjY2oRkfzHFLWN
Creating 4 Weaviate Clusters - Option 3
Country.capitalize(): Uk
The collection: Uk already exists in the Weaviate Cluster
Country.capitalize(): Wales
The collection: Wales already exists in the Weaviate Cluster
Country.capitalize(): Nothernireland
The collection: NothernIreland already exists in the Weaviate Cluster
Country.capitalize(): Scotland
The collection: Scotland already exists in the Weaviate Cluster


In [10]:
# df = pd.read_csv('./Benchmark-Data.csv')
# df

,Country,Question,Actual Answer
0,uk,In the context of the Environmental Protection...,The Environmental Land Management (ELM) scheme...
1,uk,"Under the Employment Rights Act 1996, as amend...","The Employment Rights Act 1996, as amended by ..."
2,uk,"In light of the Data Protection Act 2018, inco...",The situation described involves potential bre...
3,uk,Following the passage of the Renters' Reform B...,"The Renters' Reform Bill, expected to receive ..."
4,uk,"Under the Building Safety Act 2022, a construc...",The Building Safety Act 2022 introduces string...
5,Wales,In light of the Welsh Language (Wales) Measure...,The Welsh Language (Wales) Measure 2011 establ...
6,Wales,Considering the Environment (Wales) Act 2016 a...,The Environment (Wales) Act 2016 and the Agric...
7,Wales,"Under the Renting Homes (Wales) Act 2016, as a...","The Renting Homes (Wales) Act 2016, amended by..."
8,Wales,Following the implementation of the Additional...,The Additional Learning Needs and Education Tr...
9,Wales,In the context of the Planning (Wales) Act 201...,The Planning (Wales) Act 2015 and the Well-bei...


In [11]:
# responses = []
# docs1 = []
# docs2 = []
# docs3 = []
# for idxrow, row in tqdm(df.iterrows()):
#     country = row['Country']
#     prompt = row['Question']
#     response, docs = bot.query(query=country+' '+prompt, k=3, search_type='Vector',get_docs=True, verbose=False)
#     responses.append(response)
#     docs1.append(docs[0])
#     docs2.append(docs[1])
#     docs3.append(docs[2])

0it [00:00, ?it/s]

Collection_to_query_from: ['Uk']
The Collection: Uk is empty(0)/Not Empty(1): False


1it [02:47, 167.38s/it]

Collection_to_query_from: ['Uk']
The Collection: Uk is empty(0)/Not Empty(1): False


2it [06:30, 200.10s/it]

...

18it [58:10, 196.30s/it]

Collection_to_query_from: ['Scotland']
The Collection: Scotland is empty(0)/Not Empty(1): False


19it [1:02:30, 215.43s/it]

Collection_to_query_from: ['Uk', 'Scotland']
The Collection: Uk is empty(0)/Not Empty(1): False
The Collection: Scotland is empty(0)/Not Empty(1): False


20it [1:05:56, 197.83s/it]


In [12]:
# df['Response'] = responses
# df['Response1'] = docs1
# df['Response2'] = docs2
# df['Response3'] = docs3
# df.to_csv('./RAG_Bot_Responses_Vector_k3.csv', index=False)
# df

,Country,Question,Actual Answer,Response,Response1,Response2,Response3
0,uk,In the context of the Environmental Protection...,The Environmental Land Management (ELM) scheme...,The introduction of the ELM scheme in England ...,(2) Where the risk assessment prepared in acco...,(2) Where the risk assessment prepared in acco...,Commencement Information\nI5\nSch.\n\n2 para.\...
1,uk,"Under the Employment Rights Act 1996, as amend...","The Employment Rights Act 1996, as amended by ...","Under the amended Employment Rights Act 1996, ...",Employment Relations (Flexible Working) Act 20...,STATUTORY INSTRUMENTS\n2024\n\nNo. 438 (C. 24)...,STATUTORY INSTRUMENTS\n2023 No. 1328\nTERMS AN...
2,uk,"In light of the Data Protection Act 2018, inco...",The situation described involves potential bre...,Here's a breakdown of the legal consequences f...,(7)\n\n\nIn Article 6(1) of the GDPR (lawfulne...,Data Protection Act 2018\n2018 CHAPTER 12\nAn ...,1(1)\nModifications etc. (not altering text)\n...
3,uk,Following the passage of the Renters' Reform B...,"The Renters' Reform Bill, expected to receive ...","The abolition of Section 21 ""no-fault"" evictio...",Leasehold and Freehold Reform Act 2024\n2024 C...,STATUTORY INSTRUMENTS\n2021 No. 564\nLANDLORD ...,It introduces schedules 1 and 2 of the 2022 Ac...
4,uk,"Under the Building Safety Act 2022, a construc...",The Building Safety Act 2022 introduces string...,Here's a breakdown of how the Building Safety ...,Building Safety Act 2022\n2022 CHAPTER 30\nAn ...,Regulations 2010(3).\n\n\n(1)\nParagraphs 4A a...,STATUTORY INSTRUMENTS\n2022\n\nNo. 1210 (C. 95...
5,Wales,In light of the Welsh Language (Wales) Measure...,The Welsh Language (Wales) Measure 2011 establ...,Public bodies in Wales have a legal obligation...,17 Standards for raising awareness about Welsh...,17 Standards for raising awareness about Welsh...,Standard 39: If you have a Welsh language web ...
6,Wales,Considering the Environment (Wales) Act 2016 a...,The Environment (Wales) Act 2016 and the Agric...,The Agriculture (Wales) Act of 2023 introduces...,Agriculture (Wales) Act 2023\n2023 asc 4\nAn A...,Commencement Information\nI1\nS. 4 in force at...,Commencement Information\nI1\nS. 7 in force at...
7,Wales,"Under the Renting Homes (Wales) Act 2016, as a...","The Renting Homes (Wales) Act 2016, amended by...",## Wales Under Renting Homes (Wales) Act 2016...,(1)\nS.I. 2022/244 (W. 72).\n\nForm RHW17 is g...,There are other amendments to section 19 which...,There are other amendments to section 19 which...
8,Wales,Following the implementation of the Additional...,The Additional Learning Needs and Education Tr...,This excerpt outlines a significant legal fram...,Additional Learning Needs and Education Tribun...,This article applies to a child who on 1 Janua...,This article applies to a child who on 1 Janua...
9,Wales,In the context of the Planning (Wales) Act 201...,The Planning (Wales) Act 2015 and the Well-bei...,Here's an analysis of how Welsh planning autho...,WELSH STATUTORY INSTRUMENTS\n2021 No. 7 (W. 3)...,WELSH STATUTORY INSTRUMENTS\n2021 No. 7 (W. 3)...,WELSH STATUTORY INSTRUMENTS\n2020\n\nNo. 1216 ...


# Now Getting the Metric Values

In [16]:
df = pd.read_csv('./RAG_Bot_Responses_Vector_k3.csv')
df

,Country,Question,Actual Answer,Response,Response1,Response2,Response3
0,uk,In the context of the Environmental Protection...,The Environmental Land Management (ELM) scheme...,The introduction of the ELM scheme in England ...,(2) Where the risk assessment prepared in acco...,(2) Where the risk assessment prepared in acco...,Commencement Information\nI5\nSch.\n\n2 para.\...
1,uk,"Under the Employment Rights Act 1996, as amend...","The Employment Rights Act 1996, as amended by ...","Under the amended Employment Rights Act 1996, ...",Employment Relations (Flexible Working) Act 20...,STATUTORY INSTRUMENTS\n2024\n\nNo. 438 (C. 24)...,STATUTORY INSTRUMENTS\n2023 No. 1328\nTERMS AN...
2,uk,"In light of the Data Protection Act 2018, inco...",The situation described involves potential bre...,Here's a breakdown of the legal consequences f...,(7)\n\n\nIn Article 6(1) of the GDPR (lawfulne...,Data Protection Act 2018\n2018 CHAPTER 12\nAn ...,1(1)\nModifications etc. (not altering text)\n...
3,uk,Following the passage of the Renters' Reform B...,"The Renters' Reform Bill, expected to receive ...","The abolition of Section 21 ""no-fault"" evictio...",Leasehold and Freehold Reform Act 2024\n2024 C...,STATUTORY INSTRUMENTS\n2021 No. 564\nLANDLORD ...,It introduces schedules 1 and 2 of the 2022 Ac...
4,uk,"Under the Building Safety Act 2022, a construc...",The Building Safety Act 2022 introduces string...,Here's a breakdown of how the Building Safety ...,Building Safety Act 2022\n2022 CHAPTER 30\nAn ...,Regulations 2010(3).\n\n\n(1)\nParagraphs 4A a...,STATUTORY INSTRUMENTS\n2022\n\nNo. 1210 (C. 95...
5,Wales,In light of the Welsh Language (Wales) Measure...,The Welsh Language (Wales) Measure 2011 establ...,Public bodies in Wales have a legal obligation...,17 Standards for raising awareness about Welsh...,17 Standards for raising awareness about Welsh...,Standard 39: If you have a Welsh language web ...
6,Wales,Considering the Environment (Wales) Act 2016 a...,The Environment (Wales) Act 2016 and the Agric...,The Agriculture (Wales) Act of 2023 introduces...,Agriculture (Wales) Act 2023\n2023 asc 4\nAn A...,Commencement Information\nI1\nS. 4 in force at...,Commencement Information\nI1\nS. 7 in force at...
7,Wales,"Under the Renting Homes (Wales) Act 2016, as a...","The Renting Homes (Wales) Act 2016, amended by...",## Wales Under Renting Homes (Wales) Act 2016...,(1)\nS.I. 2022/244 (W. 72).\n\nForm RHW17 is g...,There are other amendments to section 19 which...,There are other amendments to section 19 which...
8,Wales,Following the implementation of the Additional...,The Additional Learning Needs and Education Tr...,This excerpt outlines a significant legal fram...,Additional Learning Needs and Education Tribun...,This article applies to a child who on 1 Janua...,This article applies to a child who on 1 Janua...
9,Wales,In the context of the Planning (Wales) Act 201...,The Planning (Wales) Act 2015 and the Well-bei...,Here's an analysis of how Welsh planning autho...,WELSH STATUTORY INSTRUMENTS\n2021 No. 7 (W. 3)...,WELSH STATUTORY INSTRUMENTS\n2021 No. 7 (W. 3)...,WELSH STATUTORY INSTRUMENTS\n2020\n\nNo. 1216 ...


In [25]:
Faithfulness_List = []
Answer_Relevancy_List = []
Blue_Score_List = []
Context_Precision_List = []
Context_Recall_List = []
faithfulness_with_hhem = FaithulnesswithHHEM(device='cpu', batch_size=1)
for idxdata, data in df.iterrows():
    selected_cols = df[['Response1', 'Response2', 'Response3']]
    reshaped = selected_cols.apply(lambda x: [x.values.tolist()], axis=1).values.tolist()

    data_sample = {
        'question': [data['Question']],
        'answer': [data['Response']],
        'contexts': reshaped[0],
    }
    dataset = Dataset.from_dict(data_sample)
    faithfulness_score = evaluate(dataset,metrics=[faithfulness])
    # faithfulness_score = evaluate(dataset,metrics=[faithfulness_with_hhem])
    faithfulness_score.to_pandas()
    # Faithfulness_List.append(faithfulness_score['faithfulness_with_hhem'])
    Faithfulness_List.append(faithfulness_score['faithfulness'])
    
    data_sample = {
        'question': [data['Question']],
        'answer': [data['Actual Answer']],
        'contexts': [[data['Response']]],
    }
    dataset = Dataset.from_dict(data_sample)
    answer_relavancy_score = evaluate(dataset,metrics=[answer_relevancy])
    answer_relavancy_score.to_pandas()
    Answer_Relevancy_List.append(answer_relavancy_score['answer_relevancy'])
    
    data_sample = {
        'question': [data['Question']],
        'ground_truth': [data['Actual Answer']],
        'contexts': reshaped[0],
    }
    dataset = Dataset.from_dict(data_sample)
    
    context_precision_score = evaluate(dataset,metrics=[context_precision])
    context_precision_score.to_pandas()
    Context_Precision_List.append(context_precision_score['context_precision'])
    
    context_recall_score = evaluate(dataset,metrics=[context_recall])
    context_recall_score.to_pandas()
    Context_Recall_List.append(context_recall_score['context_recall'])
    
    blue_score = sentence_bleu([data['Actual Answer'].split()], data['Response'].split())
    Blue_Score_List.append(blue_score)
    
    # print(f'faithfulness Scores: {faithfulness_score["faithfulness_with_hhem"]}')
    print(f'faithfulness Scores: {faithfulness_score["faithfulness"]}')
    print(f'answer_relavancy Scores: {answer_relavancy_score["answer_relevancy"]}')
    print(f'context_precision Scores: {context_precision_score["context_precision"]}')
    print(f'context_recall Scores: {context_recall_score["context_recall"]}')
    print(f'Blue Score: {blue_score}')
    
print(f'\nMean Value of Faithfulness: {np.mean(Faithfulness_List)}')
print(f'Mean Value of Answer Relevancy: {np.mean(Answer_Relevancy_List)}')
print(f'Mean Value of Blue Score: {np.mean(Blue_Score_List)}')
print(f'Mean Value of Context Precision: {np.mean(Context_Precision_List)}')
print(f'Mean Value of Context Recall: {np.mean(Context_Recall_List)}')

You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.
Evaluating: 100%|██████████| 1/1 [00:09<00:00,  9.33s/it]


faithfulness Scores: 0.0
answer_relavancy Scores: 0.9310266577211651
context_precision Scores: 0.9999999999666667
context_recall Scores: 0.3333333333333333
Blue Score: 0.09641440177419695


Evaluating: 100%|██████████| 1/1 [00:09<00:00,  9.06s/it]


faithfulness Scores: 0.0
answer_relavancy Scores: 0.8985107120165438
context_precision Scores: 0.0
context_recall Scores: 0.0
Blue Score: 0.044162816183707425


Evaluating: 100%|██████████| 1/1 [00:11<00:00, 11.58s/it]


...

faithfulness Scores: 0.0
answer_relavancy Scores: 0.9339965316763648
context_precision Scores: 0.0
context_recall Scores: 0.0
Blue Score: 0.05605154193207532


Evaluating: 100%|██████████| 1/1 [00:07<00:00,  8.00s/it]


faithfulness Scores: 0.1
answer_relavancy Scores: 0.9449292616345802
context_precision Scores: 0.3333333333
context_recall Scores: 0.0
Blue Score: 0.08313959165081589


Evaluating: 100%|██████████| 1/1 [00:11<00:00, 11.23s/it]


faithfulness Scores: 0.0
answer_relavancy Scores: 0.9273027020699546
context_precision Scores: 0.0
context_recall Scores: 0.0
Blue Score: 0.0606755583936463

Mean Value of Faithfulness: 0.008846153846153847
Mean Value of Answer Relevancy: 0.9200499480144231
Mean Value of Blue Score: 0.07676651001001292
Mean Value of Context Precision: 0.14999999999166666
Mean Value of Context Recall: 0.016666666666666666
